In [ ]:
import matplotlib as plt
import pandas as pd
import numpy as np
import seaborn as sb
from pathlib import Path
import matplotlib.pyplot as plot

In [ ]:
DIR = r"C:\Temp\ML_Fruad_Files"
DIR_PROTOCOL = r"C:\Temp\ML_Fruad_Files\Protocol"
DIR_Neighborhood = r"C:\Temp\ML_Fruad_Files\Neighborhood_Clusters"

In [ ]:
# creating directory and folder
folder_path = Path(DIR_PROTOCOL)
folder_path.mkdir(parents=True, exist_ok=True)

In [ ]:
df=pd.read_pickle(f'{DIR}\df_after_stage3.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29210 entries, 0 to 29209
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   gender                 29210 non-null  category
 1   lat                    29210 non-null  float64 
 2   long                   29210 non-null  float64 
 3   acct_num               29210 non-null  category
 4   amt                    29210 non-null  float64 
 5   is_fraud               29210 non-null  bool    
 6   merch_lat              29210 non-null  float64 
 7   merch_long             29210 non-null  float64 
 8   street_type            29210 non-null  category
 9   cust_cluster           29210 non-null  category
 10  num_neighborhoods      29210 non-null  int64   
 11  distance_merch_cust    29210 non-null  float64 
 12  job_category           29210 non-null  category
 13  is_Risk Manager_Job    29210 non-null  bool    
 14  category_group         29210 non-null 

In [ ]:

if 'acct_num' in df:
    summary = df.groupby('acct_num', observed=False).agg(
        num_transactions=('amt', 'count'),
        max_amount=('amt', 'max'),
        min_amount=('amt', 'min'),
    ).reset_index()

    # Count how many customers share the same pattern
    pattern_counts = summary.groupby(['num_transactions', 'max_amount', 'min_amount'], observed=False).size().reset_index(name='num_customers')

    print(pattern_counts)
# in this check it can be seen that for every data set of 'num_transactions', 'max_amount', 'min_amount'
# has one customer.
# so we can use this 3 features to identify customer instead acct_num feature

    num_transactions  max_amount  min_amount  num_customers
0                  9      982.53        5.71              1
1                343     1124.75        1.04              1
2                347     1293.58        1.00              1
3                353     1248.15        1.03              1
4                358     1221.96        1.30              1
5                358     1446.51        1.02              1
6                366     1355.38        1.04              1
7                374     1094.40        1.02              1
8                375     1792.94        1.13              1
9                386     1097.58        1.08              1
10               703     1800.25        1.01              1
11               705     1152.76        1.17              1
12               708     1868.14        1.01              1
13               715     3838.83        1.06              1
14               716     1537.89        1.03              1
15               723     1197.84        

In [ ]:
# creating 3 new feature instead acct_num
if 'acct_num' in df:
    df_new_features = df.groupby('acct_num', observed=False).agg(
        num_transactions=('amt', 'count'),
        max_amount=('amt', 'max'),
        min_amount=('amt', 'min'),
    ).reset_index()

    df = df.merge(df_new_features, on='acct_num', how='left')
    df = df.drop(columns=['acct_num'])

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29210 entries, 0 to 29209
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   gender                 29210 non-null  category
 1   lat                    29210 non-null  float64 
 2   long                   29210 non-null  float64 
 3   amt                    29210 non-null  float64 
 4   is_fraud               29210 non-null  bool    
 5   merch_lat              29210 non-null  float64 
 6   merch_long             29210 non-null  float64 
 7   street_type            29210 non-null  category
 8   cust_cluster           29210 non-null  category
 9   num_neighborhoods      29210 non-null  int64   
 10  distance_merch_cust    29210 non-null  float64 
 11  job_category           29210 non-null  category
 12  is_Risk Manager_Job    29210 non-null  bool    
 13  category_group         29210 non-null  category
 14  trans_month            29210 non-null 

In [ ]:
# Converting data into a format suitable for modeling.
if 'gender' in df:
    df['gender'] = df['gender'].astype('string')
    df['gender'] = df['gender'].replace(to_replace="F", value='1')
    df['gender'] = df['gender'].replace(to_replace="M", value='0')
    df.rename(columns={"gender": "is_female"}, inplace=True)
    df['is_female'] = df['is_female'].astype('int')

if 'street_type' in df:
    df = pd.get_dummies(df, columns=['street_type'], prefix='street')
if 'job_category' in df:
    df = pd.get_dummies(df, columns=['job_category'])
if 'category_group' in df:
    df = pd.get_dummies(df, columns=['category_group'])
if 'trans_hour_category' in df:
    df = pd.get_dummies(df, columns=['trans_hour_category'])
if 'age_group' in df:
    df = pd.get_dummies(df, columns=['age_group'], prefix='age')
if 'cust_cluster' in df:
    df = pd.get_dummies(df, columns=['cust_cluster'], prefix='cluster')
if 'trans_month' in df:
    df = pd.get_dummies(df, columns=['trans_month'], prefix='month')
if 'merchant_grouped' in df:
    df = pd.get_dummies(df, columns=['merchant_grouped'], prefix='merchant_freq')
if 'trans_day_of_week_num' in df:
    df = pd.get_dummies(df, columns=['trans_day_of_week_num'], prefix='day_of_week')

In [ ]:
df.to_pickle(f'{DIR}\df_after_stage4.pkl')
df.to_csv(f'{DIR}\Transactions_Customers_DC_2019_After_Stage4.csv', index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29210 entries, 0 to 29209
Data columns (total 75 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   is_female                            29210 non-null  int64  
 1   lat                                  29210 non-null  float64
 2   long                                 29210 non-null  float64
 3   amt                                  29210 non-null  float64
 4   is_fraud                             29210 non-null  bool   
 5   merch_lat                            29210 non-null  float64
 6   merch_long                           29210 non-null  float64
 7   num_neighborhoods                    29210 non-null  int64  
 8   distance_merch_cust                  29210 non-null  float64
 9   is_Risk Manager_Job                  29210 non-null  bool   
 10  trans_hour                           29210 non-null  int64  
 11  num_transactions            